In [15]:
sentence_a = """In truly good writing no matter how many times you read it you do not know how it is done. That is beacause there is a mystery in all great writing and that mystery does not dis-sect out. It continues and it is always valid. Each time you re-read you see or learn something new."""

sentence_b = """Good writing is true writing. If a man is making a story up it will be true in proportion to the amount of knowledge of life that he has and how conscientious he is; so that when he makes something up it is as it would truly be."""

In [16]:
from typing import List
import re


def tokenize(text: str) -> List[str]:
    text = re.sub(r"[^\w\s]", "", text)
    return text.split()

In [17]:
doc_a = tokenize(sentence_a)
doc_b = tokenize(sentence_b)

### bag of words


In [26]:
from typing import Dict
import pandas as pd


def count_terms(doc: List[str]) -> Dict:
    doc_dict = dict.fromkeys(doc, 0)

    for word in doc:
        doc_dict[word] += 1

    return doc_dict


doc_dict_a = count_terms(doc_a)
doc_dict_b = count_terms(doc_b)

freq = pd.DataFrame[doct_dict_a, doc_dict_b]
freq.T

ModuleNotFoundError: No module named 'pandas'

### TF


### IDF


### TF-IDF


In [ ]:
# term frequency
# create dict {word_1: count, word_2: count, ...}
# index dict on word, and


def tf(term: str, document: List[str]) -> int:
    return